In [2]:
import pandas as pd
from datetime import datetime

In [3]:
# Load actual data
data = pd.read_excel('./2020 dump.xlsx', sheet_name='Sheet2')

In [4]:
# Load futures data
futures_data_may = pd.read_excel('./2020 dump.xlsx', sheet_name='may')
futures_data_jun = pd.read_excel('./2020 dump.xlsx', sheet_name='jun')
futures_data_jul = pd.read_excel('./2020 dump.xlsx', sheet_name='jul')
futures_data_aug = pd.read_excel('./2020 dump.xlsx', sheet_name='aug')

In [16]:
funds_data={}
for x in range(0, 449, 9):
    fund_data = data.iloc[:,x:x+8].dropna(how="all")
    fund_name = fund_data.columns[0].split()[0]
    fund_data.columns = fund_data.iloc[0]
    fund_data['fund_name'] = fund_name
    funds_data[fund_name] = fund_data.iloc[2:,:]

In [68]:
ft_data = {'may': futures_data_may, 'jun': futures_data_jun, 'jul': futures_data_jul, 'aug': futures_data_aug}

In [70]:
ft_monthly_data = {}
for key in ft_data.keys():
    for x in range(0, 449, 9):
        fft_data = ft_data[key].iloc[:,x:x+8].dropna(how="all")
        fund_name = fft_data.columns[0].split("=")[0]
        fft_data.columns = fft_data.iloc[0]
        temp_data = fft_data.iloc[2:,:]
        
        if key == 'may':
            filter_mask = ((temp_data['Dates'] > datetime(2020, 4, 29)) & (temp_data['Dates'] < datetime(2020, 5, 28)))
        elif key == 'jun':
            filter_mask = ((temp_data['Dates'] >= datetime(2020, 5, 28)) & (temp_data['Dates'] < datetime(2020, 6, 25)))
        elif key == 'jul':
            filter_mask = ((temp_data['Dates'] >= datetime(2020, 6, 25)) & (temp_data['Dates'] < datetime(2020, 7, 30)))
        else:
            filter_mask = ((temp_data['Dates'] >= datetime(2020, 7, 30)) & (temp_data['Dates'] <= datetime(2020, 8, 27)))
        # Filter data
        temp_data = temp_data.loc[filter_mask]
        if fund_name in ft_monthly_data.keys():
            ft_monthly_data[fund_name]=ft_monthly_data[fund_name].append(temp_data, ignore_index=True)
        else:
            ft_monthly_data[fund_name] = temp_data

In [73]:
for x in ft_monthly_data:
    ft_monthly_data[x]['Close'].fillna(method = 'ffill', inplace = True)

In [76]:
ft_monthly_data['RIL']['Dates']

0       2020-04-29 09:15:00
1       2020-04-29 09:20:00
2       2020-04-29 09:25:00
3       2020-04-29 09:30:00
4       2020-04-29 09:35:00
               ...         
6295    2020-08-26 15:05:00
6296    2020-08-26 15:10:00
6297    2020-08-26 15:15:00
6298    2020-08-26 15:20:00
6299    2020-08-26 15:25:00
Name: Dates, Length: 6300, dtype: object

In [77]:
for fund in funds_data.keys():
    funds_data[fund]['Dates'] = funds_data[fund]['Dates'].astype('datetime64[ns]')

for fund in ft_monthly_data.keys():
    ft_monthly_data[fund]['Dates'] = ft_monthly_data[fund]['Dates'].astype('datetime64[ns]')

for fund in funds_data.keys():
    funds_data[fund]['Dates'] = funds_data[fund]['Dates'].astype('datetime64[ns]')

for fund in ft_monthly_data.keys():
    ft_monthly_data[fund]['Dates'] = ft_monthly_data[fund]['Dates'].astype('datetime64[ns]')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
merged_data = {}
for fund in funds_data.keys():
    if fund == 'HDFCLIFE':
        merged_data['HDFCLIFE']=pd.merge(funds_data['HDFCLIFE'], ft_monthly_data['HDFCL'], on='Dates', how='left', indicator=True)
    elif fund == 'GRASIM':
        merged_data['GRASIM']=pd.merge(funds_data['GRASIM'], ft_monthly_data['GRASI'], on='Dates', how='left', indicator=True)
    elif fund == 'ICICIBC':
        merged_data['ICICIBC']=pd.merge(funds_data['ICICIBC'], ft_monthly_data['ICICI'], on='Dates', how='left', indicator=True)
    elif fund == 'BHARTI':
        merged_data['BHARTI']=pd.merge(funds_data['BHARTI'], ft_monthly_data['BHART'], on='Dates', how='left', indicator=True)
    else:
        merged_data[fund]=pd.merge(funds_data[fund], ft_monthly_data[fund], on='Dates', how='left', indicator=True)

In [96]:
for x in merged_data:
    filter_mask = ((merged_data[x]['Dates'] > datetime(2020, 4, 28)) & (merged_data[x]['Dates'] < datetime(2020, 8, 28)))
    merged_data[x]=merged_data[x].loc[filter_mask]

In [97]:
merged_data['RIL']

,Dates,Open_x,Close_x,High_x,Low_x,Value_x,Volume_x,Number Ticks_x,fund_name,Open_y,Close_y,High_y,Low_y,Value_y,Volume_y,Number Ticks_y,_merge
3340,2020-04-28 09:05:00,1450,1450,1450,1450,214379600,147848,2,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3341,2020-04-28 09:15:00,1453.9,1443.6,1453.9,1442.95,2356426240,1627951,532,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3342,2020-04-28 09:20:00,1441.45,1418,1442.25,1412.7,2467048960,1729774,526,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3343,2020-04-28 09:25:00,1417.8,1408,1420.6,1407,1507258240,1066886,516,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3344,2020-04-28 09:30:00,1407.85,1413.45,1418.3,1406.55,1169928960,828540,498,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10385,2020-08-27 15:40:00,2110.6,2110.6,2110.6,2110.6,7.49685e+06,3552,63,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10386,2020-08-27 15:45:00,2110.6,2110.6,2110.6,2110.6,2855642,1353,35,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10387,2020-08-27 15:50:00,2110.6,2110.6,2110.6,2110.6,1.80878e+06,857,22,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10388,2020-08-27 15:55:00,2110.6,2110.6,2110.6,2110.6,1.8721e+06,887,21,RIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [98]:
for x in merged_data:
    merged_data[x]['Close_y'].fillna(method = 'ffill', inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [106]:
merged_data['RIL'][200:250]

,Dates,Open_x,Close_x,High_x,Low_x,Value_x,Volume_x,Number Ticks_x,fund_name,Open_y,Close_y,High_y,Low_y,Value_y,Volume_y,Number Ticks_y,_merge
3540,2020-04-30 12:10:00,1479.85,1486.1,1486.4,1477.35,1704994432,1150325,600,RIL,1485.1,1491.45,1491.45,1482.25,3.88725e+06,2613,283,both
3541,2020-04-30 12:15:00,1486.2,1482.85,1486.85,1480.4,967946240,652773,584,RIL,1492,1486.30,1492.2,1484.3,2.06124e+06,1386,261,both
3542,2020-04-30 12:20:00,1482.8,1492.05,1494,1482.5,1475934336,991541,536,RIL,1486.2,1496.15,1497.15,1486.2,2994713,2007,269,both
3543,2020-04-30 12:25:00,1492.75,1488.9,1494.65,1488.2,1302976896,873342,492,RIL,1496.15,1492.00,1498.8,1491.85,2.94391e+06,1968,277,both
3544,2020-04-30 12:30:00,1488.5,1484.4,1488.5,1482.25,1004427264,676382,518,RIL,1492,1486.70,1492.55,1485,1973387,1326,254,both
3545,2020-04-30 12:35:00,1483.7,1484.3,1489,1483.7,464697792,312678,520,RIL,1487.4,1487.25,1492.35,1486.5,1.31194e+06,881,205,both
3546,2020-04-30 12:40:00,1484.7,1484,1485,1482,459791808,309997,531,RIL,1486.8,1486.70,1487.75,1483.75,874986,589,179,both
3547,2020-04-30 12:45:00,1484.2,1480.15,1484.25,1480,725669952,489947,531,RIL,1486.25,1483.00,1486.8,1481.6,1050385,708,195,both
3548,2020-04-30 12:50:00,1480.45,1475.8,1480.7,1471.2,1094523776,741798,588,RIL,1483,1479.05,1483.65,1473.9,1.7692e+06,1197,250,both
3549,2020-04-30 12:55:00,1475.6,1469.55,1478,1468.95,841981504,571555,592,RIL,1479.25,1472.15,1480.85,1472,945060,640,207,both


In [107]:
for x in merged_data:
    filter_mask = ((merged_data[x]['Dates'] > datetime(2020, 5, 1)) & (merged_data[x]['Dates'] < datetime(2020, 8, 27)))
    merged_data[x] = merged_data[x].loc[filter_mask]

In [113]:
merged_data['Z'].head()

,Dates,Open_x,Close_x,High_x,Low_x,Value_x,Volume_x,Number Ticks_x,fund_name,Open_y,Close_y,High_y,Low_y,Value_y,Volume_y,Number Ticks_y,_merge
3569,2020-05-04 09:05:00,150,150,150,150,4821150,32141,2,Z,NaN,160.20,NaN,NaN,NaN,NaN,NaN,left_only
3570,2020-05-04 09:15:00,151.95,149.45,151.95,149,73855624,493156,567,Z,150.1,150.10,151.25,148.65,39506.6,263,95,both
3571,2020-05-04 09:20:00,149.45,147.25,149.45,146.75,81058448,548731,540,Z,149.5,147.70,149.5,147,31684.5,214,65,both
3572,2020-05-04 09:25:00,147.25,146.9,147.3,146,59204272,404037,550,Z,147.55,146.80,147.55,146.05,17472.2,119,45,both
3573,2020-05-04 09:30:00,146.85,147.3,147.8,146.3,41974200,285479,510,Z,147.15,147.55,147.95,146.45,14721.7,100,32,both


In [119]:
merged_data2 = {}
for x in merged_data:
    merged_data2[x] = merged_data[x][['Dates','fund_name','Close_x','Close_y']]

In [120]:
merged_data2['RIL']

,Dates,fund_name,Close_x,Close_y
3586,2020-05-04 09:05:00,RIL,1440,1466.00
3587,2020-05-04 09:15:00,RIL,1437.7,1436.35
3588,2020-05-04 09:20:00,RIL,1446.2,1445.75
3589,2020-05-04 09:25:00,RIL,1459,1460.75
3590,2020-05-04 09:30:00,RIL,1449.5,1452.00
...,...,...,...,...
10303,2020-08-26 15:40:00,RIL,2137.3,2135.70
10304,2020-08-26 15:45:00,RIL,2137.3,2135.70
10305,2020-08-26 15:50:00,RIL,2137.3,2135.70
10306,2020-08-26 15:55:00,RIL,2137.3,2135.70


In [122]:
frames = []
for key in merged_data2.keys():
    frames.append(merged_data2[key])
pd.concat(frames).to_excel('output3.xlsx')

In [121]:
with pd.ExcelWriter('output2.xlsx') as writer:
    for fund in merged_data2.keys():
        merged_data2[fund].to_excel(writer, sheet_name=fund)